# RandomForestClassifier

O principal problema das árvores de decisão é de que elas tendem ao overfitting, ou seja, se adequar quase que perfeitamente ao conjunto de dados analisado. Isso significa que elas possuem **alta variância**, i.e., podem ser criadas árvores totalmente diferentes em cima de um outro subconjunto dos dados de treinamento (ainda que possuam o mesmo Proceso Gerador de Dados).

Uma forma de reduzir isso é criar diversas árvores de decisão diferentes em cima de diversos subconjuntos dos dados de forma aleatória e com repetição (bootstrapping), selecionando inclusive as variáveis independentes de forma aleatória a fim de "descorrelacionar" os modelos ao máximo. A partir daí podemos analisar a resposta conjunta de diversos modelos, sendo este o cerne da criação das Random Forests. 

**Obs.:** A Random Forest pode ser resumida como um modelo *ensemble* de várias àrvores de decisão. Modelos *ensemble* constroem diversos modelos e analisam a sua resposta agregada como resultado da análise preditiva, e cada um possui suas peculiaridades. Em casos onde são utilizados subconjuntos aleatórios da base para treinamento, chamamos isto de bootstrapping aggregation (bagging).

Prós:

* Performance
* Alta capacidade de generalização

Contras: 
* Interpretabilidade
* Gasto computacional elevado

# Construindo o Algoritmo

### Importação das Bibliotecas e Manipulação dos Dados

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc, RocCurveDisplay
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../dados/titanic.csv")
df.drop(["PassengerId", "Name", "SibSp", "Ticket", "Cabin", "Embarked", "Parch"], axis = 1, inplace = True)
df["Sex"] = np.where(df["Sex"] == "male", 1, 0)
df.dropna(inplace = True)
df.head() # Alvo Binário: Survived

X = df.drop("Survived", axis = 1)
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

### Ajuste do Modelo

In [27]:
modelo = RandomForestClassifier(random_state = 42) # gera um modelo aleatório a cada vez que rodamos o código
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)
y_pred_train = modelo.predict(X_train)

### Métricas de Avaliação

In [28]:
# Acurácia
print(accuracy_score(y_test, y_pred))

0.8549618320610687


In [29]:
# Matriz de Confusão
print(confusion_matrix(y_test, y_pred))

[[142  20]
 [ 18  82]]


In [30]:
# Métricas de Avaliação
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.88      0.88       162
           1       0.80      0.82      0.81       100

    accuracy                           0.85       262
   macro avg       0.85      0.85      0.85       262
weighted avg       0.86      0.85      0.86       262



In [31]:
# Calculando a AUC (Area Under the Curve)
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

print("AUC - Teste :", roc_auc)

AUC - Teste : 0.8482716049382715


### Otimização

O algoritmo de Random Forest é amplamente reconhecido pela sua baixa necessidade de otimização, tendo em vista que permite estimar diferentes modelos em diferentes estágios da base de treinamento. Ainda assim, algumas escolhas de hiperparâmetros podem ser consideradas. 

Hiperparâmetros a serem otimizados:
* n_estimators -> número de árvores distintas estimadas (padrão: 10 -> pode ser muito pequeno)
* max_features -> número máximo de variáveis independentes a ser considerado em cada nó das árvores de decisão (padrão: raiz quadrada da quantidade de variáveis independentes imputada ao modelo, o que é uma boa medida)



##### Gráfico de Cotovelo

O parâmetro `n_estimators` nunca leva à queda de performance, somente aumenta o gasto computacional. Para corrigir esse problema e encontrar o melhor modelo (trade-off custo x resultado), deve-se criar o gráfico de cotovelo (Elbow Graph) e testar diferentes valores para esse hiperparâmetro.

Para isso realizamos um `GridSearch` com diversos valores e podemos plotar o atributo cv_results_ obtido, que contém certa métrica de avaliação escolhida (geralmente ['mean_test_score']) para cada modelo testado.

##### Feature Importances

Para cada árvore de decisão estimada no RandomForest, o modelo realiza a divisão do conjunto de dados tendo como objetivo a máxima redução de impuridade. Tendo em vista que são estimadas diversas árvores com diversos subconjuntos tanto de observações quanto de variáveis independentes, pode-se calcular a média de redução de impuridade (mean decrease impurity) que cada variável proporciona.

Com isso, o modelo pode comparar a importância relativa de cada variável independente em conceder capacidade preditiva ao modelo, sendo assim uma ótima forma de se analisar a qualidade das variáveis testadas.

Tudo isso é fornecido pelo atributo `feature_importances_` do modelo.

**Obs.:** Quando o RandomForest realiza tarefas de regressão, é utilizada a variância dos resultados para calcular a importância relativa de uma variável.

In [32]:
modelo.feature_importances_

array([0.05751806, 0.45942431, 0.23445662, 0.248601  ])

In [35]:
# Colocando as variáveis em ordem crescente
imp = pd.DataFrame({'Variáveis' : X_train.columns, 'Importância' : modelo.feature_importances_})
imp.sort_values(by = ['Importância'], ascending = False)

,Variáveis,Importância
1,Sex,0.459424
3,Fare,0.248601
2,Age,0.234457
0,Pclass,0.057518
